In [ ]:
import matplotlib
matplotlib.use('tkagg')
from pyimzml.ImzMLParser import ImzMLParser
import numpy as np
import plotly.graph_objects as go
import os
from scipy.signal import find_peaks
import pandas as pd
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import time
from collections import defaultdict
from scipy.ndimage import rotate
from scipy.signal import savgol_filter, detrend
import plotly.io as pio
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk

pio.renderers.default = "notebook" # Revert to a backend suitable for saving

class MSIVisualizerTkinter:
    def __init__(self):
        """Initialize the visualizer with GUI file selection and peak threshold control using Tkinter."""
        self.root = tk.Tk()
        self.root.title("MSI Data Visualizer")

        self.file_path = None
        self.parser = None
        self.full_mz = None
        self.avg_intensity = None
        self.original_avg_intensity = None
        self.mz_min = 0
        self.mz_max = 0
        self.current_mz = 0
        self.normalize = tk.BooleanVar(value=False)
        self.show_labels = tk.BooleanVar(value=False)
        self.intensity_range = [0, 1000]
        self.peak_threshold_value = 1.0e3
        self.spectrum_plot_type = tk.StringVar(value='line')
        self.current_colormap = tk.StringVar(value='viridis')
        self.current_rotation_angle = tk.DoubleVar(value=0)
        self.label_color = tk.StringVar(value='black')
        self.label_fontsize = tk.IntVar(value=12)
        self.peak_properties = tk.StringVar(value='m/z')
        self.baseline_correction = tk.StringVar(value='None')
        self.noise_reduction = tk.StringVar(value='None')

        self.progress_var = tk.DoubleVar(value=0.0)
        self.progress_label_var = tk.StringVar(value="Initializing...")

        self.fig_msi = None
        self.canvas_msi = None
        self.fig_spectrum = None
        self.canvas_spectrum = None

        self.create_widgets()

    def format_file_size(self, size_bytes):
        """Formats file size in human-readable units."""
        if size_bytes < 1024:
            return f"{size_bytes} bytes"
        elif size_bytes < 1024**2:
            return f"{size_bytes / 1024:.2f} KB"
        elif size_bytes < 1024**3:
            return f"{size_bytes / 1024**2:.2f} MB"
        else:
            return f"{size_bytes / 1024**3:.2f} GB"

    def update_progress(self, value=None, description=None):
        """Update the progress bar and status."""
        if value is not None:
            self.progress_var.set(value)
        if description is not None:
            self.progress_label_var.set(description)
        self.root.update_idletasks()

    def calculate_average_spectrum_with_progress(self):
        """Calculates the average mass spectrum with progress tracking (optimized)."""
        mz_intensity_sums = defaultdict(float)
        spectrum_counts = defaultdict(int)

        total_spectra = len(self.parser.coordinates)
        update_interval = max(1, total_spectra // 20)
        last_update_time = time.time()

        for i in range(total_spectra):
            try:
                if i % update_interval == 0:
                    current_time = time.time()
                    if current_time - last_update_time > 1:
                        progress = (i + 1) / total_spectra
                        self.update_progress(value=progress, description=f"Processing spectrum {i + 1}/{total_spectra}")
                        last_update_time = current_time

                mz_values, intensities = self.parser.getspectrum(i)
                for mz, intensity in zip(mz_values, intensities):
                    mz_intensity_sums[mz] += intensity
                    spectrum_counts[mz] += 1

            except Exception as e:
                print(f"Warning: Skipping spectrum {i} due to error: {str(e)}")
                continue

        combined_mz = np.array(sorted(mz_intensity_sums.keys()))
        avg_intensity = np.array([mz_intensity_sums[mz] / spectrum_counts[mz] for mz in combined_mz])

        self.update_progress(value=1.0, description="Spectrum processing complete!")
        return combined_mz, avg_intensity

    def get_mz_range(self):
        """Get the m/z range from the first spectrum."""
        try:
            mz_values, _ = self.parser.getspectrum(0)
            return min(mz_values), max(mz_values)
        except Exception as e:
            raise RuntimeError(f"Failed to get m/z range: {str(e)}")

    def generate_image_data(self, mz_value, normalize=False):
        """Generate image data efficiently."""
        print(f"Generating image data with normalize={normalize}")
        x_coords, y_coords = zip(*[(x, y) for x, y, _ in self.parser.coordinates])
        intensity_map = []

        mz_mask = (mz_value - 0.1, mz_value + 0.1)

        for i, _ in enumerate(self.parser.coordinates):
            try:
                mz_values, intensities = self.parser.getspectrum(i)
                mask = (mz_values >= mz_mask[0]) & (mz_values <= mz_mask[1])
                if np.any(mask):
                    intensity = np.sum(intensities[mask])
                    if normalize:
                        tic = np.sum(intensities)
                        intensity = intensity / tic if tic > 0 else 0
                else:
                    intensity = 0
                intensity_map.append(intensity)

            except Exception as e:
                print(f"Warning: Skipping spectrum {i} due to error: {str(e)}")
                intensity_map.append(0)

        return np.array(x_coords), np.array(y_coords), np.array(intensity_map)

    def update_plot(self):
        """Update the MSI image visualization."""
        if self.parser is None:
            return

        if self.fig_msi:
            self.fig_msi.clf()
        else:
            self.fig_msi = Figure(figsize=(8, 6), dpi=100)
            self.canvas_msi = FigureCanvasTkAgg(self.fig_msi, master=self.msi_frame)
            self.canvas_msi.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        ax = self.fig_msi.add_subplot(111)
        x, y, intensity = self.generate_image_data(self.current_mz, self.normalize.get())
        y_coords_corrected = np.max(y) - y

        center_x = (np.max(x) + np.min(x)) / 2
        center_y = (np.max(y_coords_corrected) + np.min(y_coords_corrected)) / 2
        angle_rad = np.radians(self.current_rotation_angle.get())
        rotated_x = (x - center_x) * np.cos(angle_rad) - (y_coords_corrected - center_y) * np.sin(angle_rad) + center_x
        rotated_y = (x - center_x) * np.sin(angle_rad) + (y_coords_corrected - center_y) * np.cos(angle_rad) + center_y

        min_intensity, max_intensity = self.intensity_range
        intensity[intensity < min_intensity] = 0
        intensity[intensity > max_intensity] = 0

        scatter = ax.scatter(rotated_x, rotated_y, c=intensity, cmap=self.current_colormap.get(), marker='s', s=100)
        ax.set_title(f'MSI Image for m/z {self.current_mz:.2f}')
        ax.set_xlabel('X Coordinate')
        ax.set_ylabel('Y Coordinate')
        ax.invert_yaxis()
        self.fig_msi.colorbar(scatter, ax=ax, label='Intensity')
        self.canvas_msi.draw()

    def update_spectrum_plot(self):
        """Updates the spectrum visualization based on selected plot type and preprocessing."""
        if self.full_mz is None or self.avg_intensity is None:
            return

        processed_intensity = self.apply_spectrum_preprocessing(self.original_avg_intensity.copy())

        if self.fig_spectrum:
            self.fig_spectrum.clf()
        else:
            self.fig_spectrum = Figure(figsize=(8, 6), dpi=100)
            self.canvas_spectrum = FigureCanvasTkAgg(self.fig_spectrum, master=self.spectrum_frame)
            self.canvas_spectrum.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
            self.toolbar_spectrum = NavigationToolbar2Tk(self.canvas_spectrum, self.spectrum_frame)
            self.toolbar_spectrum.update()
            self.canvas_spectrum._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        ax = self.fig_spectrum.add_subplot(111)
        plot_type = self.spectrum_plot_type.get()

        if plot_type == 'line':
            ax.plot(self.full_mz, processed_intensity)
        elif plot_type == 'area':
            ax.fill_between(self.full_mz, processed_intensity)
        elif plot_type == 'centroid':
            for mz, intensity in zip(self.full_mz, processed_intensity):
                ax.vlines(mz, 0, intensity, colors='blue', linewidth=1)

        ax.set_title('Average Mass Spectrum')
        ax.set_xlabel('m/z')
        ax.set_ylabel('Intensity')
        ax.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
        self.canvas_spectrum.draw()
        self.on_peak_labels_change()

    def apply_spectrum_preprocessing(self, intensity_data):
        """Applies selected preprocessing steps to the spectrum intensity data."""
        processed_intensity = intensity_data.copy()

        baseline_method = self.baseline_correction.get()
        if baseline_method == 'Detrend':
            processed_intensity = detrend(processed_intensity)

        noise_reduction_method = self.noise_reduction.get()
        if noise_reduction_method == 'Savitzky-Golay':
            try:
                processed_intensity = savgol_filter(processed_intensity, window_length=5, polyorder=3)
            except Exception as e:
                print(f"Warning: Savitzky-Golay filter failed: {e}. Skipping noise reduction.")

        return processed_intensity

    def update_intensity_range_slider(self):
        """Updates the range of the intensity slider based on current image."""
        if self.parser is None:
            return
        x, y, intensity = self.generate_image_data(self.current_mz, self.normalize.get())
        max_intensity = np.max(intensity) if intensity.size > 0 else 1000
        self.intensity_lower_scale.config(to=max_intensity, resolution=max_intensity / 100 if max_intensity > 0 else 10)
        self.intensity_upper_scale.config(to=max_intensity, resolution=max_intensity / 100 if max_intensity > 0 else 10)
        if self.normalize.get():
            self.intensity_range = [0, 1]
            self.intensity_lower_scale.set(0)
            self.intensity_upper_scale.set(1)
        else:
            self.intensity_range = [0, max_intensity]
            self.intensity_lower_scale.set(0)
            self.intensity_upper_scale.set(max_intensity)

    def on_mz_change(self, event):
        """Handles m/z slider and text input changes."""
        try:
            new_mz_value = float(self.mz_entry.get())
            if self.mz_min <= new_mz_value <= self.mz_max:
                self.current_mz = new_mz_value
                self.mz_slider.set(self.current_mz)
                self.update_plot()
        except ValueError:
            pass

    def on_mz_slider_change(self, value):
        """Handles m/z slider changes."""
        try:
            self.current_mz = float(value)
            self.mz_entry.delete(0, tk.END)
            self.mz_entry.insert(0, f"{self.current_mz:.2f}")
            self.update_plot()
        except ValueError:
            pass

    def on_norm_change(self):
        """Handles TIC normalization toggle."""
        self.update_intensity_range_slider()
        self.update_plot()

    def on_intensity_range_change(self, event):
        """Handles intensity range slider changes."""
        lower = self.intensity_lower_scale.get()
        upper = self.intensity_upper_scale.get()
        if lower > upper:
            lower, upper = upper, lower
            self.intensity_lower_scale.set(lower)
            self.intensity_upper_scale.set(upper)
        self.intensity_range = [lower, upper]
        self.update_plot()

    def on_colormap_change(self, *args):
        """Handles colormap dropdown changes."""
        self.update_plot()

    def on_rotation_change(self, value):
        """Handles rotation slider changes."""
        try:
            self.current_rotation_angle.set(float(value))
            self.update_plot()
        except ValueError:
            pass

    def on_label_toggle(self):
        """Handles label toggle button click."""
        self.update_peak_labels()

    def on_peak_threshold_change(self, value):
        """Handles peak threshold slider changes."""
        try:
            self.peak_threshold_value = float(value)
            self.on_peak_labels_change()
        except ValueError:
            pass

    def on_spectrum_plot_type_change(self, *args):
        """Handles spectrum plot type dropdown change."""
        self.update_spectrum_plot()

    def on_reset_zoom_clicked(self):
        """Handles reset zoom button click."""
        if self.fig_spectrum and self.full_mz is not None:
            ax = self.fig_spectrum.axes[0]
            ax.set_xlim(min(self.full_mz), max(self.full_mz))
            self.canvas_spectrum.draw()

    def on_peak_label_change(self, *args):
        """Handles changes to peak label customization widgets."""
        self.update_peak_labels()

    def on_preprocess_change(self, *args):
        """Handles changes to spectrum preprocessing dropdowns."""
        self.update_spectrum_plot()

    def on_peak_labels_change(self):
        """Updates peak labels on the spectrum plot."""
        if self.fig_spectrum and self.full_mz is not None and self.avg_intensity is not None:
            ax = self.fig_spectrum.axes[0]
            for annotation in ax.texts:
                if annotation.get_text().startswith("m/z:"):
                    annotation.set_visible(False)

            if self.show_labels.get():
                xlim = ax.get_xlim()
                mask = (self.full_mz >= xlim[0]) & (self.full_mz <= xlim[1])
                mz_in_view = self.full_mz[mask]
                intensity_in_view = self.apply_spectrum_preprocessing(self.original_avg_intensity[mask])

                if not mz_in_view.size:
                    return

                peak_indices = find_peaks(intensity_in_view, height=self.peak_threshold_value)[0]

                for index in peak_indices:
                    peak_mz = mz_in_view[index]
                    peak_intensity = intensity_in_view[index]
                    label_text = f"m/z: {peak_mz:.2f}"
                    if "intensity" in self.peak_properties.get():
                        label_text += f"\nIntensity: {peak_intensity:.2e}"
                    ax.text(peak_mz, peak_intensity, label_text, color=self.label_color.get(), fontsize=self.label_fontsize.get(), ha='center', va='bottom')
            self.canvas_spectrum.draw()

    def find_peaks_in_view(self):
        """Finds peaks in the average spectrum that are within the current x-axis range."""
        if self.fig_spectrum and self.full_mz is not None and self.avg_intensity is not None:
            ax = self.fig_spectrum.axes[0]
            xlim = ax.get_xlim()
            mask = (self.full_mz >= xlim[0]) & (self.full_mz <= xlim[1])
            mz_in_view = self.full_mz[mask]
            intensity_in_view = self.apply_spectrum_preprocessing(self.original_avg_intensity[mask])

            if not mz_in_view.size:
                return []

            peak_indices = find_peaks(intensity_in_view, height=self.peak_threshold_value)[0]

            peaks = []
            for index in peak_indices:
                peak_mz = mz_in_view[index]
                peak_intensity = intensity_in_view[index]
                peaks.append((peak_mz, peak_intensity))
            return peaks
        return []

    def on_spectrum_peak_click_handler(self, event):
        """Handles clicks on peaks in the spectrum plot."""
        if event.inaxes == self.fig_spectrum.axes[0]:
            try:
                x, y = event.xdata, event.ydata
                if self.full_mz is not None:
                    closest_index = np.argmin(np.abs(self.full_mz - x))
                    clicked_mz = self.full_mz[closest_index]
                    self.current_mz = clicked_mz
                    self.mz_entry.delete(0, tk.END)
                    self.mz_entry.insert(0, f"{self.current_mz:.2f}")
                    self.mz_slider.set(self.current_mz)
                    self.update_plot()
            except TypeError:
                pass

    def save_msi_image(self):
        """Save the MSI image as a PNG file."""
        if self.fig_msi:
            filepath = filedialog.asksaveasfilename(
                defaultextension=".png",
                filetypes=[("PNG files", "*.png"), ("All files", "*.*")],
                title="Save MSI Image As"
            )
            if filepath:
                try:
                    self.fig_msi.savefig(filepath)
                    messagebox.showinfo("Save Successful", f"MSI image saved to {filepath}")
                except Exception as e:
                    messagebox.showerror("Save Error", f"Error saving MSI image: {e}")
        else:
            messagebox.showerror("Error", "No MSI image to save.")

    def save_spectrum(self):
        """Save the mass spectrum as a PNG file."""
        if self.fig_spectrum:
            filepath = filedialog.asksaveasfilename(
                defaultextension=".png",
                filetypes=[("PNG files", "*.png"), ("All files", "*.*")],
                title="Save Spectrum As"
            )
            if filepath:
                try:
                    self.fig_spectrum.savefig(filepath)
                    messagebox.showinfo("Save Successful", f"Mass spectrum saved to {filepath}")
                except Exception as e:
                    messagebox.showerror("Save Error", f"Error saving mass spectrum: {e}")
        else:
            messagebox.showerror("Error", "No mass spectrum to save.")

    def load_file(self):
        """Load the imzML file and initialize data."""
        self.file_path = filedialog.askopenfilename(
            title="Select imzML file",
            filetypes=[("imzML files", "*.imzml")]
        )
        if self.file_path:
            try:
                self.progress_bar.pack(pady=10)
                self.progress_label.pack()
                self.root.update_idletasks()

                file_size_bytes = os.path.getsize(self.file_path)
                file_size_formatted = self.format_file_size(file_size_bytes)
                print(f"File size: {file_size_formatted}")

                self.update_progress(description="Loading imzML file...")
                self.parser = ImzMLParser(self.file_path)
                coord_count = len(self.parser.coordinates)
                print(f"Number of spectra: {coord_count}")

                self.update_progress(description="Loading spectra coordinates...")
                time.sleep(0.1)

                self.mz_min, self.mz_max = self.get_mz_range()
                self.mz_slider.config(from_=self.mz_min, to=self.mz_max, resolution=(self.mz_max - self.mz_min) / 1000)
                self.current_mz = (self.mz_max - self.mz_min) / 2
                self.mz_slider.set(self.current_mz)
                self.mz_entry.delete(0, tk.END)
                self.mz_entry.insert(0, f"{self.current_mz:.2f}")

                self.update_progress(description="Calculating average spectrum...")
                self.full_mz, self.avg_intensity = self.calculate_average_spectrum_with_progress()
                self.original_avg_intensity = self.avg_intensity.copy()

                self.update_plot()
                self.update_spectrum_plot()
                self.update_intensity_range_slider()

                self.progress_bar.pack_forget()
                self.progress_label.pack_forget()

            except FileNotFoundError:
                tk.messagebox.showerror("Error", "No file selected.")
            except Exception as e:
                tk.messagebox.showerror("Error", f"Failed to load file: {e}")

    def create_widgets(self):
        """Create the Tkinter widgets."""
        # Progress Bar
        self.progress_var = tk.DoubleVar(value=0.0)
        self.progress_bar = ttk.Progressbar(self.root, variable=self.progress_var, maximum=1.0)
        self.progress_label_var = tk.StringVar(value="Ready")
        self.progress_label = tk.Label(self.root, textvariable=self.progress_label_var)

        # Menu Bar
        menubar = tk.Menu(self.root)
        filemenu = tk.Menu(menubar, tearoff=0)
        filemenu.add_command(label="Open", command=self.load_file)
        filemenu.add_command(label="Save MSI Image As...", command=self.save_msi_image)
        filemenu.add_command(label="Save Spectrum As...", command=self.save_spectrum)
        filemenu.add_separator()
        filemenu.add_command(label="Exit", command=self.root.quit)
        menubar.add_cascade(label="File", menu=filemenu)
        self.root.config(menu=menubar)

        # Main Tabbed Interface
        self.tab_control = ttk.Notebook(self.root)

        # MSI Image Tab
        self.msi_tab = ttk.Frame(self.tab_control)
        self.tab_control.add(self.msi_tab, text='MSI Image')
        self.msi_tab.grid_columnconfigure(0, weight=1)
        self.msi_tab.grid_rowconfigure(0, weight=1)
        self.msi_frame = ttk.Frame(self.msi_tab)
        self.msi_frame.grid(row=0, column=0, sticky="nsew")

        # MSI Image Controls
        controls_frame_msi = ttk.LabelFrame(self.msi_tab, text='MSI Image Controls')
        controls_frame_msi.grid(row=1, column=0, padx=10, pady=10, sticky="ew")

        ttk.Label(controls_frame_msi, text="m/z:").grid(row=0, column=0, padx=5, pady=5)
        self.mz_entry = ttk.Entry(controls_frame_msi, width=10)
        self.mz_entry.grid(row=0, column=1, padx=5, pady=5)
        self.mz_entry.bind('<Return>', self.on_mz_change)

        self.mz_slider = tk.Scale(controls_frame_msi, from_=self.mz_min, to=self.mz_max, orient=tk.HORIZONTAL,
                                  label="m/z Slider", command=self.on_mz_slider_change)
        self.mz_slider.grid(row=1, column=0, columnspan=2, padx=5, pady=5, sticky='ew')

        self.norm_checkbox_widget = ttk.Checkbutton(controls_frame_msi, text="TIC Normalization", variable=self.normalize, command=self.on_norm_change)
        self.norm_checkbox_widget.grid(row=0, column=2, padx=5, pady=5)

        ttk.Label(controls_frame_msi, text="Colormap:").grid(row=2, column=0, padx=5, pady=5)
        self.colormap_combo = ttk.Combobox(controls_frame_msi, textvariable=self.current_colormap,
                                            values=['viridis', 'plasma', 'rainbow', 'magma', 'greys', 'hot', 'cividis', 'jet', 'electric'])
        self.colormap_combo.grid(row=2, column=1, padx=5, pady=5)
        self.colormap_combo.bind("<<ComboboxSelected>>", self.on_colormap_change)

        ttk.Label(controls_frame_msi, text="Rotation (°):").grid(row=3, column=0, padx=5, pady=5)
        self.rotation_slider = tk.Scale(controls_frame_msi, from_=0, to=360, orient=tk.HORIZONTAL,
                                        label="Rotation", command=self.on_rotation_change)
        self.rotation_slider.grid(row=3, column=1, columnspan=2, padx=5, pady=5, sticky='ew')

        intensity_frame = ttk.LabelFrame(controls_frame_msi, text='Intensity Range')
        intensity_frame.grid(row=4, column=0, columnspan=3, padx=5, pady=5, sticky='ew')
        self.intensity_lower_scale = tk.Scale(intensity_frame, from_=0, to=1000, orient=tk.HORIZONTAL, label="Min", command=self.on_intensity_range_change)
        self.intensity_lower_scale.pack(side=tk.LEFT, fill=tk.X, expand=True)
        self.intensity_upper_scale = tk.Scale(intensity_frame, from_=0, to=1000, orient=tk.HORIZONTAL, label="Max", command=self.on_intensity_range_change)
        self.intensity_upper_scale.pack(side=tk.LEFT, fill=tk.X, expand=True)

        # Spectrum Tab
        self.spectrum_tab = ttk.Frame(self.tab_control)
        self.tab_control.add(self.spectrum_tab, text='Mass Spectrum')
        self.spectrum_frame = ttk.Frame(self.spectrum_tab)
        self.spectrum_frame.pack(fill=tk.BOTH, expand=True)

        # Spectrum Controls
        controls_frame_spectrum = ttk.LabelFrame(self.spectrum_tab, text='Spectrum Controls')
        controls_frame_spectrum.pack(padx=10, pady=10, fill=tk.X)

        ttk.Checkbutton(controls_frame_spectrum, text="Show m/z Labels", variable=self.show_labels, command=self.on_label_toggle).grid(row=0, column=0, padx=5, pady=5)

        ttk.Label(controls_frame_spectrum, text="Peak Threshold:").grid(row=1, column=0, padx=5, pady=5)
        self.peak_threshold_slider = tk.Scale(controls_frame_spectrum, from_=0, to=10000, orient=tk.HORIZONTAL,
                                             label="Threshold", command=self.on_peak_threshold_change)
        self.peak_threshold_slider.grid(row=1, column=1, padx=5, pady=5, sticky='ew')
        self.peak_threshold_slider.set(self.peak_threshold_value)

        ttk.Label(controls_frame_spectrum, text="Plot Type:").grid(row=0, column=1, padx=5, pady=5)
        self.spectrum_plot_type_combo = ttk.Combobox(controls_frame_spectrum, textvariable=self.spectrum_plot_type,
                                                      values=['line', 'area', 'centroid'])
        self.spectrum_plot_type_combo.grid(row=0, column=2, padx=5, pady=5)
        self.spectrum_plot_type_combo.bind("<<ComboboxSelected>>", self.on_spectrum_plot_type_change)

        ttk.Button(controls_frame_spectrum, text="Reset Zoom", command=self.on_reset_zoom_clicked).grid(row=0, column=3, padx=5, pady=5)

        ttk.Label(controls_frame_spectrum, text="Label Color:").grid(row=2, column=0, padx=5, pady=5)
        self.label_color_combo = ttk.Combobox(controls_frame_spectrum, textvariable=self.label_color,
                                               values=['black', 'red', 'blue', 'green'])
        self.label_color_combo.grid(row=2, column=1, padx=5, pady=5)
        self.label_color_combo.bind("<<ComboboxSelected>>", self.on_peak_label_change)

        ttk.Label(controls_frame_spectrum, text="Label Size:").grid(row=2, column=2, padx=5, pady=5)
        self.label_fontsize_scale = tk.Scale(controls_frame_spectrum, from_=8, to=20, orient=tk.HORIZONTAL,
                                             label="Size", command=lambda v: self.label_fontsize.set(int(v)))
        self.label_fontsize_scale.grid(row=2, column=3, padx=5, pady=5, sticky='ew')
        self.label_fontsize_scale.set(self.label_fontsize.get())

        ttk.Label(controls_frame_spectrum, text="Label Props:").grid(row=3, column=0, padx=5, pady=5)
        self.peak_properties_combo = ttk.Combobox(controls_frame_spectrum, textvariable=self.peak_properties,
                                                  values=['m/z', 'm/z + intensity'])
        self.peak_properties_combo.grid(row=3, column=1, padx=5, pady=5)
        self.peak_properties_combo.bind("<<ComboboxSelected>>", self.on_peak_label_change)

        ttk.Label(controls_frame_spectrum, text="Baseline Corr:").grid(row=4, column=0, padx=5, pady=5)
        self.baseline_correction_combo = ttk.Combobox(controls_frame_spectrum, textvariable=self.baseline_correction,
                                                       values=['None', 'Detrend'])
        self.baseline_correction_combo.grid(row=4, column=1, padx=5, pady=5)
        self.baseline_correction_combo.bind("<<ComboboxSelected>>", self.on_preprocess_change)

        ttk.Label(controls_frame_spectrum, text="Noise Reduct:").grid(row=4, column=2, padx=5, pady=5)
        self.noise_reduction_combo = ttk.Combobox(controls_frame_spectrum, textvariable=self.noise_reduction,
                                                    values=['None', 'Savitzky-Golay'])
        self.noise_reduction_combo.grid(row=4, column=3, padx=5, pady=5)
        self.noise_reduction_combo.bind("<<ComboboxSelected>>", self.on_preprocess_change)

        # Peak Table Tab (Currently Empty)
        self.peak_table_tab = ttk.Frame(self.tab_control)
        self.tab_control.add(self.peak_table_tab, text='Peak Table')
        ttk.Label(self.peak_table_tab, text="Peak table functionality will be added here.").pack(padx=10, pady=10)

        self.tab_control.pack(expand=1, fill="both")

    def run(self):
        """Run the Tkinter application."""
        self.root.mainloop()

if __name__ == '__main__':
    try:
        visualizer = MSIVisualizerTkinter()
        visualizer.run()
    except FileNotFoundError:
        print("No file selected. Visualizer initialization cancelled.")
    except RuntimeError as e:
        print(f"Initialization error: {e}")